In [312]:
from dataclasses import dataclass, field
from enum import Enum, auto, StrEnum
from typing import Optional, Dict, Self, Generic, TypeVar, List, Iterator, NamedTuple, Protocol
from collections import namedtuple


class EmitsGcode(Protocol):
    def gcode(self) -> str:
        ...

class CommandType(Enum):
    GO_HOME = auto()
    GO_XY = auto()


class CommandInfo(NamedTuple):
    gcode: str
    fields: List[str]
    description: Optional[str] = None


_command_to_info: Dict[CommandType, CommandInfo] = {
    CommandType.GO_HOME: CommandInfo("G28", [], "Go to home position"),
    CommandType.GO_XY: CommandInfo("G1", ["x", "y", "z"], "Go to specified XY position"),
}


Numeric = TypeVar("Numeric", int, float)


# @dataclass
# class Coordinate:
#     x: Optional[Numeric] = None
#     y: Optional[Numeric] = None
#     z: Optional[Numeric] = None


class Coordinate(NamedTuple):
    x: Optional[Numeric] = None
    y: Optional[Numeric] = None
    z: Optional[Numeric] = None

    def gcode(self) -> Iterator[str]:
        return " ".join([f"{v}" for _, v in self._asdict().items() if v is not None])


_hardcoded_locations: Dict[str, Coordinate] = {
    "tiprack": Coordinate(20, 100, 20),
    "trash": Coordinate(0, 0, 0),
    "home": Coordinate(100, 100, 0),
}


class NamedLocation(StrEnum):
    TIPRACK = "tiprack"
    TRASH = "trash"
    HOME = "home"


def HardcodedLocation(name: NamedLocation) -> Coordinate:
    return _hardcoded_locations[name]


@dataclass
class GcodeCommand(EmitsGcode):
    type: CommandType
    arg: Optional[Coordinate] = None

    def __post_init__(self):
      try:
        if self.arg is not None:
          assert len(self.arg) == len(_command_to_info[self.type].fields)
        else:
          assert len(_command_to_info[self.type].fields) == 0
      except AssertionError:
        raise ValueError(f"Command {self.type} expects {len(_command_to_info[self.type].fields)} arguments, but got {len(self.arg)}")

    def gcode(self) -> str:
        return f"{_command_to_info[self.type].gcode} {self.arg.gcode() if self.arg else ''}"


@dataclass
class CommandSequence(EmitsGcode):
    seq : list[GcodeCommand] = field(default_factory=list[GcodeCommand])

    def __len__(self) -> int:
        return len(self.seq)

    def __add__(self, gc: GcodeCommand) -> Self:
        self.seq.append(gc)
        return self

    def __iter__(self) -> Iterator[GcodeCommand]:
        return iter(self.seq)

    def gcode(self) -> str:
        return "\n".join([gc.gcode() for gc in self.seq])


pick_tip = CommandSequence([
    GcodeCommand(type=CommandType.GO_XY,
                 arg=HardcodedLocation(NamedLocation.TIPRACK)),
    GcodeCommand(type=CommandType.GO_XY, arg=HardcodedLocation(NamedLocation.HOME)),
    GcodeCommand(type=CommandType.GO_HOME),
])


go_home = GcodeCommand(type=CommandType.GO_HOME)

print(go_home.gcode())
print(pick_tip.gcode())

G28 
G1 20 100 20
G1 100 100 0
G28 


In [313]:
def run_through_serial_till_complete(gcode: str) -> None:
    # print(gcode)
    ...

In [314]:
def execute(cmd: CommandSequence) -> None:
  print("Execution Start")

  try:
    for index, gc in enumerate(cmd):
      print(f" | Executing Cmd#{index}: {gc.gcode()}")
      import time
      time.sleep(1)
      run_through_serial_till_complete(gc.gcode())

    print("Execution Complete")
  except (KeyboardInterrupt) as e:
    print(f"Execution Failed\n{e}")

In [315]:
execute(pick_tip)
print(pick_tip)

Execution Start
 | Executing Cmd#0: G1 20 100 20
 | Executing Cmd#1: G1 100 100 0
 | Executing Cmd#2: G28 
Execution Complete
CommandSequence(seq=[GcodeCommand(type=<CommandType.GO_XY: 2>, arg=Coordinate(x=20, y=100, z=20)), GcodeCommand(type=<CommandType.GO_XY: 2>, arg=Coordinate(x=100, y=100, z=0)), GcodeCommand(type=<CommandType.GO_HOME: 1>, arg=None)])
